In [2]:
# train 1000장 미만 출력

In [1]:
import os
import yaml
from collections import defaultdict

# =======================================
# ✅ 경로 수정 (데이터 폴더까지 포함!)
# =======================================
yaml_path = "./food_data_v8/data.yaml"
train_labels_dir = "./food_data_v8/train/labels"

# =======================================
# ✅ YAML 로드
# =======================================
with open(yaml_path, 'r') as f:
    data_yaml = yaml.safe_load(f)

class_names = data_yaml['names']

# =======================================
# ✅ 클래스별 이미지 수 세기
# =======================================
class_counts = defaultdict(set)

for label_file in os.listdir(train_labels_dir):
    if not label_file.endswith(".txt"):
        continue
    with open(os.path.join(train_labels_dir, label_file), 'r') as f:
        for line in f:
            class_id = int(line.strip().split()[0])
            image_id = label_file.replace(".txt", "")
            class_counts[class_id].add(image_id)

# =======================================
# ✅ 출력
# =======================================
print(f"{'ClassID':<8} | {'Name':<20} | {'Count'}")
print("-"*40)

for class_id, image_set in sorted(class_counts.items()):
    count = len(image_set)
    if count < 1000:
        name = class_names[class_id] if class_id < len(class_names) else "Unknown"
        print(f"{class_id:<8} | {name:<20} | {count}")

ClassID  | Name                 | Count
----------------------------------------
0        | almond               | 51
1        | apple                | 950
2        | asparagus            | 60
8        | blackberry           | 37
9        | blueberry            | 102
16       | cherry               | 66
19       | egg                  | 33
22       | grape                | 188
23       | green bean           | 96
24       | green onion          | 78
26       | kiwi                 | 74
27       | lemon                | 269
29       | lime                 | 92
30       | mandarin             | 27
31       | mushroom             | 329
33       | orange               | 649
36       | peach                | 63
37       | pear                 | 82
38       | pineapple            | 154
42       | raspberry            | 64
43       | strawberry           | 279
46       | watermelon           | 93


In [ ]:
# train 1000장 미만 클래스 삭제

In [3]:
import os

EXCLUDE_CLASSES = [0, 1, 2, 8, 9, 16, 19, 22, 23, 24, 26, 27, 29, 30, 31, 33, 36, 37, 38, 42, 43, 46]

# ============================================================
# ✅ 여러 폴더 한 번에 처리 (복사본 폴더 기준)
# ============================================================
dataset_dir = "./food_data_v8_delete"
sets = ['train', 'valid', 'test']

total_deleted = 0

for split in sets:
    labels_dir = os.path.join(dataset_dir, split, 'labels')
    images_dir = os.path.join(dataset_dir, split, 'images')

    deleted_count = 0

    for label_file in os.listdir(labels_dir):
        if not label_file.endswith(".txt"):
            continue

        label_path = os.path.join(labels_dir, label_file)

        with open(label_path, 'r') as f:
            lines = f.readlines()

        # 제외 클래스만 제거
        new_lines = []
        for line in lines:
            class_id = int(line.strip().split()[0])
            if class_id not in EXCLUDE_CLASSES:
                new_lines.append(line)

        if not new_lines:
            # ✔️ 라벨 내용 다 빠졌으면 라벨+이미지 삭제
            os.remove(label_path)
            image_jpg = os.path.join(images_dir, label_file.replace('.txt', '.jpg'))
            image_png = os.path.join(images_dir, label_file.replace('.txt', '.png'))

            if os.path.exists(image_jpg):
                os.remove(image_jpg)
            elif os.path.exists(image_png):
                os.remove(image_png)

            deleted_count += 1
        else:
            # ✔️ 일부만 제거 → 다시 저장
            with open(label_path, 'w') as f:
                f.writelines(new_lines)

    print(f"✅ [{split}] 삭제된 이미지+라벨 쌍: {deleted_count}개")
    total_deleted += deleted_count

print(f"✅ 전체 처리 완료! 삭제된 총 파일 쌍: {total_deleted}개")

✅ [train] 삭제된 이미지+라벨 쌍: 1726개
✅ [valid] 삭제된 이미지+라벨 쌍: 249개
✅ [test] 삭제된 이미지+라벨 쌍: 162개
✅ 전체 처리 완료! 삭제된 총 파일 쌍: 2137개


In [4]:
import os
from collections import defaultdict

# ============================================================
# ✅ 1) 제외할 클래스 리스트
# ============================================================
EXCLUDE_CLASSES = [0, 1, 2, 8, 9, 16, 19, 22, 23, 24, 26, 27, 29, 30, 31, 33, 36, 37, 38, 42, 43, 46]

# ============================================================
# ✅ 2) 데이터셋 폴더 경로 & 세트 리스트
# ============================================================
dataset_dir = "./food_data_v8_delete"
sets = ['train', 'valid', 'test']

# ============================================================
# ✅ 3) 각 세트별 클래스별 잔여 이미지 개수 확인
# ============================================================
for split in sets:
    labels_dir = os.path.join(dataset_dir, split, 'labels')
    class_counts = defaultdict(set)  # 클래스ID : 이미지ID set

    for label_file in os.listdir(labels_dir):
        if not label_file.endswith(".txt"):
            continue

        label_path = os.path.join(labels_dir, label_file)
        with open(label_path, 'r') as f:
            lines = f.readlines()

        for line in lines:
            class_id = int(line.strip().split()[0])
            if class_id in EXCLUDE_CLASSES:
                image_id = label_file.replace(".txt", "")
                class_counts[class_id].add(image_id)

    print(f"\n✅ [{split.upper()}] 세트의 제외 대상 클래스 잔여 이미지 수:")
    print("-" * 40)
    found = False
    for class_id in sorted(class_counts.keys()):
        count = len(class_counts[class_id])
        if count > 0:
            print(f"ClassID: {class_id:<3} | Count: {count}")
            found = True
    if not found:
        print("✅ 깨끗함! 제외 클래스 없음.")

print("\n✅ 점검 완료!")



✅ [TRAIN] 세트의 제외 대상 클래스 잔여 이미지 수:
----------------------------------------
✅ 깨끗함! 제외 클래스 없음.

✅ [VALID] 세트의 제외 대상 클래스 잔여 이미지 수:
----------------------------------------
✅ 깨끗함! 제외 클래스 없음.

✅ [TEST] 세트의 제외 대상 클래스 잔여 이미지 수:
----------------------------------------
✅ 깨끗함! 제외 클래스 없음.

✅ 점검 완료!


In [5]:
# 삭제된 데이터셋으로 yolo v8m 모델 학습

In [6]:
# [A] 처음 실행할 때 전체 코드

# ============================================================
# ✅ 1) 필수 라이브러리
# ============================================================
import os
import pandas as pd
from ultralytics import YOLO

# ============================================================
# ✅ 2) YOLOv8m 모델 로드 (사전 학습 weight)
# ============================================================
model = YOLO('yolov8m.pt')  # 처음 실행할 땐 pretrained 모델로!

# ============================================================
# ✅ 3) 학습 실행
# ============================================================
results = model.train(
    data="./food_data_v8_delete/data.yaml",  # ✅ 데이터셋 경로
    epochs=100,                              # ✅ 총 에포크 수
    imgsz=640,                               # ✅ 이미지 사이즈
    batch=16,                                # ✅ 배치 사이즈 (너 GPU에 맞게 조절)
    device=0,                                # ✅ 첫 번째 GPU 사용
    resume=False                             # ✅ 새로 시작할 땐 반드시 False
)

# ============================================================
# ✅ 4) 학습 로그 저장 경로 확인 + CSV 불러오기
# ============================================================
results_csv = os.path.join(results.save_dir, "results.csv")

print(f"\n✅ [저장된 CSV 경로]: {results_csv}\n")

df = pd.read_csv(results_csv)
display(df)

# ============================================================
# ✅ 5) 에포크별 검증 지표 출력
# ============================================================
print("\n✅ [에포크별 지표 요약]")
print(df[['epoch', 'metrics/precision(B)', 'metrics/recall(B)', 
          'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'metrics/f1(B)']])


100%|██████████| 49.7M/49.7M [00:04<00:00, 10.8MB/s]


Ultralytics 8.3.163  Python-3.11.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./food_data_v8_delete/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=

100%|██████████| 755k/755k [00:00<00:00, 11.4MB/s]

Overriding model.yaml nc=80 with nc=47

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytic

 21                  -1  2   4207104  ultralytics.nn.modules.block.C2f             [960, 576, 2]                 
 22        [15, 18, 21]  1   3802909  ultralytics.nn.modules.head.Detect           [47, [192, 384, 576]]         
Model summary: 169 layers, 25,883,533 parameters, 25,883,517 gradients, 79.2 GFLOPs

Transferred 469/475 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 137/137 [00:36<00:00,  3.79it/s]


                   all       4370      35887      0.922      0.818      0.878      0.735
               avocado        350       1039      0.951      0.926      0.964      0.824
                banana        184       5285      0.784      0.448      0.578      0.364
                 beans        403        883      0.976      0.954      0.981      0.883
                  beet        317        897      0.956      0.975       0.99      0.866
           bell pepper        375       1393      0.912      0.752      0.822      0.703
              broccoli        405       1641      0.802      0.692      0.789      0.599
      brussels sprouts        241        597      0.959      0.933       0.98      0.839
               cabbage        293        547      0.956      0.973      0.991      0.888
                carrot        554       2773      0.855      0.628      0.751      0.554
           cauliflower        294        961      0.961      0.905      0.948      0.826
                celer

,epoch,time,train/box_loss,train/cls_loss,train/dfl_loss,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),val/box_loss,val/cls_loss,val/dfl_loss,lr/pg0,lr/pg1,lr/pg2
0,1,766.782,1.27440,2.40941,1.46033,0.59909,0.48694,0.52136,0.33393,1.23601,1.44054,1.36757,0.003332,0.003332,0.003332
1,2,5225.070,1.21527,1.67860,1.38387,0.61841,0.49101,0.53245,0.33953,1.25384,1.43967,1.40050,0.006599,0.006599,0.006599
2,3,6568.070,1.26956,1.72891,1.42226,0.56891,0.44610,0.46251,0.28775,1.32120,1.62248,1.45629,0.009800,0.009800,0.009800
3,4,7200.290,1.28995,1.75096,1.44188,0.68328,0.52439,0.58729,0.37924,1.26129,1.30435,1.41016,0.009703,0.009703,0.009703
4,5,10936.900,1.25391,1.58063,1.41690,0.72166,0.58702,0.65442,0.42910,1.21843,1.13386,1.38547,0.009604,0.009604,0.009604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,232910.000,0.66030,0.40357,1.00228,0.92139,0.81717,0.87743,0.73309,0.74072,0.45448,0.96581,0.000595,0.000595,0.000595
96,97,237511.000,0.65253,0.39807,0.99998,0.92137,0.81767,0.87760,0.73369,0.73942,0.45398,0.96512,0.000496,0.000496,0.000496
97,98,240836.000,0.64988,0.39426,0.99944,0.92151,0.81774,0.87787,0.73431,0.73808,0.45371,0.96443,0.000397,0.000397,0.000397
98,99,242273.000,0.64411,0.39082,0.99712,0.92144,0.81802,0.87794,0.73471,0.73684,0.45352,0.96390,0.000298,0.000298,0.000298



✅ [에포크별 지표 요약]


KeyError: "['metrics/f1(B)'] not in index"

In [ ]:
# [B] 중간에 끊겼을 때 이어서 실행할 전체 코드

# ============================================================
# ✅ 1) 필수 라이브러리
# ============================================================
import os
import pandas as pd
from ultralytics import YOLO

# ============================================================
# ✅ 2) 중단된 모델 weight 불러오기 (last.pt)
# ============================================================
model = YOLO('runs/train/exp/weights/last.pt')  # ✅ 경로는 너의 exp에 맞게!

# ============================================================
# ✅ 3) 이어서 학습하기
# ============================================================
results = model.train(
    data="./food_data_v8_delete/data.yaml",  # ✅ 동일한 데이터셋
    epochs=100,                              # ✅ 총 목표 에포크 수 (이전 진행분까지 포함)
    imgsz=640,
    batch=16,
    device=0,
    resume=True                              # ✅ 이어서 학습할 땐 반드시 True
)

# ============================================================
# ✅ 4) CSV 확인
# ============================================================
results_csv = os.path.join(results.save_dir, "results.csv")

print(f"\n✅ [저장된 CSV 경로]: {results_csv}\n")

df = pd.read_csv(results_csv)
display(df)

# ============================================================
# ✅ 5) 에포크별 검증 지표 출력
# ============================================================
print("\n✅ [에포크별 지표 요약]")
print(df[['epoch', 'metrics/precision(B)', 'metrics/recall(B)', 
          'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'metrics/f1(B)']])
